In [1]:
import matplotlib.pyplot as plt

import networkx as nx

import numpy as np

# For reading files
import os

# Data manipulation
import pandas as pd

# Text cleaning
import re

import requests

## Text processing
# NLP
import spacy
from spacy.lang.en import English
from spacy.matcher import Matcher
from spacy.tokens import Span
nlp = spacy.load("en_core_web_sm")

# Sentence boundary detection
import pysbd

# More NLP for fun
from textblob import TextBlob

from tqdm import tqdm

# Pipeline creation
from sklearn.pipeline import Pipeline

# Text extraction
from tika import parser

pd.set_option('display.max_colwidth', 200)
%matplotlib inline

In [2]:
# Wanted to see details
pd.options.display.max_rows = 999

## Get the cover letter df did

In [95]:

# read_file = pd.read_excel (r"C:\Users\sambe\Projects\Cover_Letter_Analysis\data\Data_tobecompiled\Media_Planner_New York_NY_2020-06-10\Cover_Letter\Media_Planner_New York_NY_2020-06-10 2.csv")
# read_file.to_csv ('cover_letters.csv', index = None, header=True)
cover_letter = pd.read_csv(r"C:\Users\sambe\Projects\Cover_Letter_Analysis\data\Data_tobecompiled\Research_Assistant_New_York_NY_2020-06-19\Cover_Letter\Research_Assistant_New_York_NY_2020-06-19.csv")

In [96]:
cover_letter

,Research_Assistant_New_York_NY_2020-06-19,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,NaN,NaN,NaN,NaN,NaN
1,Jun 19,NaN,NaN,NaN,NaN
2,Stephanie Dunson,"To Whom It May Concern,\n\nThis letter accompanied by my resume in answer to your advertisement on indeed.com, I am interested in being considered for the position entitled “Research Assistant"".\n...",NaN,NaN,NaN
3,Donna Tiu,"Hello,\n\nAs a Summa cum laude honors graduate with a B.S. in Biology from San Diego State University (SDSU), I possess extensive experience in data evaluation, scientific research, data analytics...",NaN,NaN,NaN
4,Chuck Chen,"To whom it may concern,\n\n\nI would like to apply for the Research Assistant position at Rookieplay.\n\nAs a graduate student at the University of Pennsylvania studying Public Administration, I l...",NaN,NaN,NaN
...,...,...,...,...,...
74,Kendra Holloway,"Hello,\n\nI am a New York City based arts administrator with a great amount of experience in the areas of transcription, social media management, video production, and high level administrative su...",NaN,NaN,NaN
75,Lauren Dean,"June 5, 2020\nDear Rookieplay Hiring Manager:\nI am excited to apply for the Research Assistant position advertised on your website. I have a\nPhD in sociology (2017), as well as Master’s and Bach...",NaN,NaN,NaN
76,Justin Shaw,"Dear Hiring Manager:\nI am excited to be applying for the position Research Assistant at Rookieplay. As someone with a keen interest in research and analysis, the process of gathering and examinin...",NaN,NaN,NaN
77,Irene Wazgowska,"(404) 441-9492 wazgowip@gmail.com\nDear Rookieplay Hiring Team,\nAs a brand builder, someone curious by nature, and a skilled listener, the opportunity to join Rookieplay in the\nResearch Assistan...",NaN,NaN,NaN


In [108]:
# Pull the data from the numberous cover letter files. 

more_cover_letter = pd.DataFrame(columns=['username', 'raw_cover_letters'])

all_text = []
filenames = []


def extract_data():
    for filename in os.listdir(r'C:\Users\sambe\Projects\Cover_Letter_Analysis\data\Data_tobecompiled\Research_Assistant_New_York_NY_2020-06-19\Cover_Letter\Add.CoverLetters'):
        parsed = parser.from_file(r'C:\Users\sambe\Projects\Cover_Letter_Analysis\data\Data_tobecompiled\Research_Assistant_New_York_NY_2020-06-19\Cover_Letter\Add.CoverLetters\\' + filename)
        text = parsed['content']
        if parsed['content'] == None:
            print(filename + " is bad.")
        filenames.append(filename)
#         resumes_df.username = filenames
        try:
            text = text.replace('\n', '')          
        except:
            pass
        all_text.append(text)
#         print("extracting data from " + filename)
    more_cover_letter.raw_cover_letters = all_text
    more_cover_letter.username = filenames
#     resumes_df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
    
extract_data()

In [115]:
more_cover_letter = pd.read_csv('more_cover_letter.csv', index_col=0)

In [26]:
cover_letter.to_csv('cover_letter.csv')

In [29]:
cover_letter = pd.read_csv('cover_letter.csv', index_col=0)

In [116]:
more_cover_letter

,username,raw_cover_letters
0,Benjamin Drury,"Microsoft Word - Drury - CLDear Application Review Committee, As colleges and universities are preparing for the future of education, many of my colleagues are talking about worst case scenarios..."
1,Bradford Shin,"Microsoft Word - cover letter rookieplayBradford Shin brshin@davidson.edu, (516) 996-1015 / Syosset, NY June 9, 2020 Rookieplay Inc. New York, NY To whom it may concern, I am writing to apply ..."
2,Danielle Tavasti,"DANIELLE TAVASTI (310) 525-4625 | danielle.tavasti@yahoo.com June 6th, 2020 RookiePlay-NewYork, NY RE: Research Assistant Dear Hiring Manager: I write to excitedly express my interest in..."
3,Elyse Echegaray,"Elyse M. Echegaray290 5th St. Brooklyn, NY | Elysemeche@gmail.com | 347-387-2363 RookieplayJune 7, 2020 New York, New YorkDear Hiring Manager:With my relevant research, organizational, and analyti..."
4,Irene Wazgowska,"Microsoft Word - Irene Wazgowska - CL Research Assistant RookieplayIrene P. Wazgowska (404) 441-9492 wazgowip@gmail.com Dear Rookieplay Hiring Team, As a brand builder, someone..."
5,Joon Choe,"Joon Choe8250 E Harvard Ave. Denver, CO, 80231630 290 5114hyungjoon.choe@gmail.comJune 6th, 2020Dear Hiring Manager, When the opportunity at Rookieplay arrested my attention, I have become excited..."
6,Julia Potter,"Julia Potter Cover Letter RookieplayJULIA POTTER 301 Edgevale Road, Baltimore, MD 21210 • Julia.R.Potter.20@Dartmouth.edu • 443-761-1836 June 8, 2020 Rookieplay Dear Hiring Committee: I am a..."
7,Katie Bent-Koerick,"cover letter rookieplay9 June 2020 Dear Hiring Manager, I am eager to apply for the Research Assistant position here at Rookieplay. As a graduate student with extensive experience in quantitative ..."
8,Lauren Dean,"Lauren Dean cover letter research assistant June 5, 2020 Dear Rookieplay Hiring Manager: I am excited to apply for the Research Assistant position advertised on your website. I have a PhD in ..."
9,Melissa Manley,"1050 N Stuart St Arlington, VA 22201703-508-4098 | mmanley3@umd.edu June 5, 2020To whom it may concern, I am writing to you to apply for the Research Assistant position at Rookieplay. I am current..."


In [110]:
more_cover_letter = more_cover_letter.replace(to_replace = '.pdf', value = '', regex = True)
more_cover_letter = more_cover_letter.replace(to_replace = '.docx', value = '', regex = True)
more_cover_letter = more_cover_letter.replace(to_replace = ' Letter of Recommendation.pdf', value = '', regex = True)

In [111]:
more_cover_letter['username'] = more_cover_letter['username'].str.replace(r"([A-Z])", r" \1").str.strip()

In [123]:
# Drops extra columns. We only want username and text columns
cover_letter = cover_letter.drop(cover_letter.columns[1], axis=1)

In [112]:
more_cover_letter.to_csv('more_cover_letter.csv')

In [126]:
# Drops first two rows, which is usually NaN and the date. 
# Renames the columns to username and raw_cover_letters
# Orders the usernames alphabetically and resets the index

# cover_letter = cover_letter.drop([0,1], axis=0)
# cover_letter.columns = ['username', 'raw_cover_letters']
cover_letter = cover_letter.sort_values('username').reset_index(drop=True)

In [124]:
# Creates a column in between for the title of the position they applied for. Cleans the text by deleting the \n

cover_letter.insert(1, 'title', 'Research Assistant')
cover_letter = cover_letter.replace(to_replace = '\n', value = '', regex = True)

In [25]:
# Check to see the completed df all clean!

cover_letter.username[-1]

'Kendra Brown'

In [127]:
cover_letter

,username,title,raw_cover_letters
0,Adaora Aroh,Research Assistant,"My name is Adaora Aroh and I am a recent master’s degree graduate in Biochemistry and Molecular Biology, I served as a student research assistant during my program and I have almost 2 years of res..."
1,Alec Getz,Research Assistant,"I’m writing to express my interest in applying for the position of research assistant (as posted on indeed.com). Although I don't have a sociology degree, I graduated cum laude in philosophy from ..."
2,Ali Munk,Research Assistant,"Although I do not have exact experience for this position. In my previous role at Infinite CBD, I did lots of research on the human body, the endocannabinoid system, and how the cannabis plant in ..."
3,Amenaghawon Oyegun,Research Assistant,"Dear Rookieplay,The Research Assistant role with your organization aligns with my past career roles in research and public health. I am applying for this role because I value Rookieplay’s commitme..."
4,Anna Ferguson,Research Assistant,"June 8, 2020RookiePlayNew York, New YorkTo whom it may concern,I am writing to you regarding the Research Assistant position currently available with Rookieplay. With a background in Anthropology,..."
...,...,...,...
74,Stephanie Simms,Research Assistant,"June 8, 2020To Whom It May Concern:I would like to express my interest at Rookieplay as a Research Assistant. As a graduate in Psychology and Spanish Language from James Madison University and a M..."
75,Tess Tucker,Research Assistant,"To Whom It May Concern,My name is Tess Tucker, and I have a strong background in executive office administration and management in a telecommuting environment. I love Rookieplay’s mission to help ..."
76,Tyler Leli,Research Assistant,To Whom It May Concern:I was thrilled to learn about the available Research Assistant positions with Rookieplay. I believe that I am a well-qualified candidate for this opportunity. Hopefully you ...
77,Veronique Calmels,Research Assistant,I was thrilled when I saw a posting for a Research Assistant position. I have always been passionate about psychology and its biological influences. Throughout my college and professional career I...


In [117]:
cover_letter = cover_letter.append(more_cover_letter)

In [119]:
cover_letter.drop_duplicates(subset ="username", 
                     keep = 'first', inplace = True) 

In [120]:
len(cover_letter)

79

In [128]:
cover_letter

,username,title,raw_cover_letters
0,Adaora Aroh,Research Assistant,"My name is Adaora Aroh and I am a recent master’s degree graduate in Biochemistry and Molecular Biology, I served as a student research assistant during my program and I have almost 2 years of res..."
1,Alec Getz,Research Assistant,"I’m writing to express my interest in applying for the position of research assistant (as posted on indeed.com). Although I don't have a sociology degree, I graduated cum laude in philosophy from ..."
2,Ali Munk,Research Assistant,"Although I do not have exact experience for this position. In my previous role at Infinite CBD, I did lots of research on the human body, the endocannabinoid system, and how the cannabis plant in ..."
3,Amenaghawon Oyegun,Research Assistant,"Dear Rookieplay,The Research Assistant role with your organization aligns with my past career roles in research and public health. I am applying for this role because I value Rookieplay’s commitme..."
4,Anna Ferguson,Research Assistant,"June 8, 2020RookiePlayNew York, New YorkTo whom it may concern,I am writing to you regarding the Research Assistant position currently available with Rookieplay. With a background in Anthropology,..."
...,...,...,...
74,Stephanie Simms,Research Assistant,"June 8, 2020To Whom It May Concern:I would like to express my interest at Rookieplay as a Research Assistant. As a graduate in Psychology and Spanish Language from James Madison University and a M..."
75,Tess Tucker,Research Assistant,"To Whom It May Concern,My name is Tess Tucker, and I have a strong background in executive office administration and management in a telecommuting environment. I love Rookieplay’s mission to help ..."
76,Tyler Leli,Research Assistant,To Whom It May Concern:I was thrilled to learn about the available Research Assistant positions with Rookieplay. I believe that I am a well-qualified candidate for this opportunity. Hopefully you ...
77,Veronique Calmels,Research Assistant,I was thrilled when I saw a posting for a Research Assistant position. I have always been passionate about psychology and its biological influences. Throughout my college and professional career I...


## Get that resume df did

In [129]:
# Construct blank dataframe with the columns needed
resumes_df = pd.DataFrame(columns=['username', 'raw_resume'])
resumes_df

all_text = []
filenames = []




# Pull the data from the numberous resume files. 

def extract_data():
    directory = r"C:\Users\sambe\Projects\Cover_Letter_Analysis\data\Data_tobecompiled\Research_Assistant_New_York_NY_2020-06-19"
    for filename in os.listdir(directory):
        parsed = parser.from_file(directory + "\\" + filename)
        text = parsed['content']
        if parsed['content'] == None:
            print(filename + " is bad.")
        filenames.append(filename)
#         resumes_df.username = filenames
        try:
            text = text.replace('\n', '')          
        except:
            pass
        all_text.append(text)
#         print("extracting data from " + filename)
    resumes_df.raw_resume = all_text
    resumes_df.username = filenames
#     resumes_df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
    
extract_data()

In [130]:
len(all_text)

193

In [131]:
resumes_df

,username,raw_resume
0,ResumeAbraBurkett.pdf,"Indeed ResumeAbra BurkettBrooklyn, NY 11206abraburkett3_pyd@indeedemail.com(310) 351-6405Authorized to work in the US for any employerWork ExperienceSeasonalSix Flags America - Bowie, MDMarch 2019..."
1,ResumeAbubakarKhan.pdf,A B U B A K A R M . K H A N CURRENT DESIGNATION Currently a Medical Graduate in the process of completing the USMLE. ECFMG Certification: -Step 1: PASSED (1st attempt) -Step 2: Pending EDUCA...
2,ResumeAdaoraAroh.pdf,"Indeed ResumeAdaora ArohOpen to the following job roles; Research assistant, Research Technician, ResearchAssociate, Clinical research coordinator, Project coordinatorAstoria, NY 11106adaoraaroh8_..."
3,ResumeAlecGetz.pdf,"Indeed ResumeAlec GetzNew York, NY 10003alecgetz6_7o2@indeedemail.com917-657-1098Work ExperiencePractice Office AssociateFlexStaff - New York, NYSeptember 2019 to Present• Met with patients to acq..."
4,ResumeAliMunk.pdf,"Indeed ResumeAli MunkContent Queen with a Business Savvy MindDenver, CO 80246allisonmunk4_hxx@indeedemail.com7193933732Motivated, young creative with proven experience in planning and executing ev..."
...,...,...
188,ResumeYolandaClark.doc,"YOLANDA CLARKPROFESSIONAL SUMMARY· 5+ years high volume disaster relief call center, customer service, clerical/administrative support experience· Expert in customer care/communications, relations..."
189,ResumeYUANYUANJYYJIANG.docx,"YUANYUAN JIANG218-17 73 Ave., Unit 1 Bayside, NY, USA 11364 (347)213-5507 jyy2269@gmail.comPermanent Work Authorization • No Visa Sponsorship RequiredEDUCATION PhD in Marketing ..."
190,ResumeYvonneNyarko.docx,"Yvonne Nyarko2160 E Tremont AveBronx, NY 10462646-633-9067Ynyarko2@gmail.comEDUCATIONNortheast Ohio Medical University (NEOMED)\t, Rootstown, OH\tGraduate Class of 2021· Pharmacy program Pharm.D.T..."
191,ResumeZainabMohammed.pdf,"Indeed ResumeZainab MohammedINSTRUCTORSan Antonio, TX 78238zainabamohammed8_dtu@indeedemail.com210-842-6892Work ExperienceINSTRUCTORUNIVERSITY OF TEXAS SAN ANTONIOSeptember 2019 to Present6 credit..."


In [137]:
# Drops the first row, which is usually the spreadsheet of cover letters

resumes_df = resumes_df.drop(resumes_df.index[0])

In [132]:
# Cleans the dataframe of useless drivel including file extensions, Indeed, numbers. Feel free to scan through the 
# usernames and add more!

resumes_df = resumes_df.replace(to_replace = 'Resume', value = '', regex = True)
resumes_df = resumes_df.replace(to_replace = 'Indeed', value = '', regex = True)
resumes_df = resumes_df.replace(to_replace = '.docx', value = '', regex = True)
resumes_df = resumes_df.replace(to_replace = ' \(1\)', value = '', regex = True)
resumes_df = resumes_df.replace(to_replace = '.pdf', value = '', regex = True)
resumes_df = resumes_df.replace(to_replace = '.doc', value = '', regex = True)
resumes_df['username'] = resumes_df['username'].str.replace(r"([A-Z])", r" \1").str.strip()

In [133]:
resumes_df

,username,raw_resume
0,Abra Burkett,"Abra BurkettBrooklyn, NY 11206abraburkett3_pyd@indeedemail.com(310) 351-6405Authorized to work in the US for any employerWork ExperienceSeasonalSix Flags America - Bowie, MDMarch 2019 to PresentS..."
1,Abubakar Khan,A B U B A K A R M . K H A N CURRENT DESIGNATION Currently a Medical Graduate in the process of completing the USMLE. ECFMG Certification: -Step 1: PASSED (1st attempt) -Step 2: Pending EDUCA...
2,Adaora Aroh,"Adaora ArohOpen to the following job roles; Research assistant, Research Technician, ResearchAssociate, Clinical research coordinator, Project coordinatorAstoria, NY 11106adaoraaroh8_m5t@indeedem..."
3,Alec Getz,"Alec GetzNew York, NY 10003alecgetz6_7o2@indeedemail.com917-657-1098Work ExperiencePractice Office AssociateFlexStaff - New York, NYSeptember 2019 to Present• Met with patients to acquire insuran..."
4,Ali Munk,"Ali MunkContent Queen with a Business Savvy MindDenver, CO 80246allisonmunk4_hxx@indeedemail.com7193933732Motivated, young creative with proven experience in planning and executing events, digita..."
...,...,...
188,Yolanda Clark,"YOLANDA CLARKPROFESSIONAL SUMMARY· 5+ years high volume disaster relief call center, customer service, clerical/administrative support experience· Expert in customer care/communications, relations..."
189,Y U A N Y U A N J Y Y J I A N G,"YUANYUAN JIANG218-17 73 Ave., Unit 1 Bayside, NY, USA 11364 (347)213-5507 jyy2269@gmail.comPermanent Work Authorization • No Visa Sponsorship RequiredEDUCATION PhD in Marketing ..."
190,Yvonne Nyarko,"Yvonne Nyarko2160 E Tremont AveBronx, NY 10462646-633-9067Ynyarko2@gmail.comEDUCATIONNortheast Ohio Medical University (NEOMED)\t, Rootstown, OH\tGraduate Class of 2021· Pharmacy program Pharm.D.T..."
191,Zainab Mohammed,"Zainab MohammedINSTRUCTORSan Antonio, TX 78238zainabamohammed8_dtu@indeedemail.com210-842-6892Work ExperienceINSTRUCTORUNIVERSITY OF TEXAS SAN ANTONIOSeptember 2019 to Present6 credit hours • Pr..."


In [134]:
# Save as csv to be further edited in google sheets.

resumes_df.to_csv('resumes.csv')

In [135]:
resumes = pd.read_csv('resumes.csv', index_col=0)

In [136]:
resumes

,username,raw_resume
0,Abra Burkett,"Abra BurkettBrooklyn, NY 11206abraburkett3_pyd@indeedemail.com(310) 351-6405Authorized to work in the US for any employerWork ExperienceSeasonalSix Flags America - Bowie, MDMarch 2019 to PresentS..."
1,Abubakar Khan,A B U B A K A R M . K H A N CURRENT DESIGNATION Currently a Medical Graduate in the process of completing the USMLE. ECFMG Certification: -Step 1: PASSED (1st attempt) -Step 2: Pending EDUCA...
2,Adaora Aroh,"Adaora ArohOpen to the following job roles; Research assistant, Research Technician, ResearchAssociate, Clinical research coordinator, Project coordinatorAstoria, NY 11106adaoraaroh8_m5t@indeedem..."
3,Alec Getz,"Alec GetzNew York, NY 10003alecgetz6_7o2@indeedemail.com917-657-1098Work ExperiencePractice Office AssociateFlexStaff - New York, NYSeptember 2019 to Present• Met with patients to acquire insuran..."
4,Ali Munk,"Ali MunkContent Queen with a Business Savvy MindDenver, CO 80246allisonmunk4_hxx@indeedemail.com7193933732Motivated, young creative with proven experience in planning and executing events, digita..."
...,...,...
188,Yolanda Clark,"YOLANDA CLARKPROFESSIONAL SUMMARY· 5+ years high volume disaster relief call center, customer service, clerical/administrative support experience· Expert in customer care/communications, relations..."
189,YuanYuan Jiang,"YUANYUAN JIANG218-17 73 Ave., Unit 1 Bayside, NY, USA 11364 (347)213-5507 jyy2269@gmail.comPermanent Work Authorization • No Visa Sponsorship RequiredEDUCATION PhD in Marketing ..."
190,Yvonne Nyarko,"Yvonne Nyarko2160 E Tremont AveBronx, NY 10462646-633-9067Ynyarko2@gmail.comEDUCATIONNortheast Ohio Medical University (NEOMED)\t, Rootstown, OH\tGraduate Class of 2021· Pharmacy program Pharm.D.T..."
191,Zainab Mohammed,"Zainab MohammedINSTRUCTORSan Antonio, TX 78238zainabamohammed8_dtu@indeedemail.com210-842-6892Work ExperienceINSTRUCTORUNIVERSITY OF TEXAS SAN ANTONIOSeptember 2019 to Present6 credit hours • Pr..."


In [137]:
# Resort the names and reset the index

resumes = resumes.sort_values('username').reset_index(drop=True)

In [138]:
# Merge the cover letter and resume dataframes, matching the common names

compiled_df = cover_letter.merge(resumes, left_on='username', right_on='username')

In [139]:
compiled_df

,username,title,raw_cover_letters,raw_resume
0,Adaora Aroh,Research Assistant,"My name is Adaora Aroh and I am a recent master’s degree graduate in Biochemistry and Molecular Biology, I served as a student research assistant during my program and I have almost 2 years of res...","Adaora ArohOpen to the following job roles; Research assistant, Research Technician, ResearchAssociate, Clinical research coordinator, Project coordinatorAstoria, NY 11106adaoraaroh8_m5t@indeedem..."
1,Alec Getz,Research Assistant,"I’m writing to express my interest in applying for the position of research assistant (as posted on indeed.com). Although I don't have a sociology degree, I graduated cum laude in philosophy from ...","Alec GetzNew York, NY 10003alecgetz6_7o2@indeedemail.com917-657-1098Work ExperiencePractice Office AssociateFlexStaff - New York, NYSeptember 2019 to Present• Met with patients to acquire insuran..."
2,Ali Munk,Research Assistant,"Although I do not have exact experience for this position. In my previous role at Infinite CBD, I did lots of research on the human body, the endocannabinoid system, and how the cannabis plant in ...","Ali MunkContent Queen with a Business Savvy MindDenver, CO 80246allisonmunk4_hxx@indeedemail.com7193933732Motivated, young creative with proven experience in planning and executing events, digita..."
3,Amenaghawon Oyegun,Research Assistant,"Dear Rookieplay,The Research Assistant role with your organization aligns with my past career roles in research and public health. I am applying for this role because I value Rookieplay’s commitme...","Amenaghawon Oyegun Austin, Texas ▪ (682) 472-5689 ▪ sere1009@gmail.com\tProfessional ExperienceTexas Health and Human Services Civil Rights Office\t\t\t\t \t\tAustin, Texas 40 hou..."
4,Anna Ferguson,Research Assistant,"June 8, 2020RookiePlayNew York, New YorkTo whom it may concern,I am writing to you regarding the Research Assistant position currently available with Rookieplay. With a background in Anthropology,...","Anna FergusonLansdale, PA 19446annaferguson725_e4s@indeedemail.com2159332315Experienced Marketing Assistant with a demonstrated history of working in the individual and familyservices industry. S..."
...,...,...,...,...
73,Stephanie Simms,Research Assistant,"June 8, 2020To Whom It May Concern:I would like to express my interest at Rookieplay as a Research Assistant. As a graduate in Psychology and Spanish Language from James Madison University and a M...","Stephanie M. Simmssmsimms17@gmail.comEnglish and Spanish speakingEDUCATIONUniversidad de Málaga, October 2017 – October 2019· Master’s of Applied Sociology· Thesis: Analysis of Volunteers in the S..."
74,Tess Tucker,Research Assistant,"To Whom It May Concern,My name is Tess Tucker, and I have a strong background in executive office administration and management in a telecommuting environment. I love Rookieplay’s mission to help ...","Tess TuckerVirtual Executive Assistant & Marketing SpecialistPensacola, FL 32504tesstucker7_eqh@indeedemail.com(850) 898-4004I have 10+ years of experience in the customer service, managerial, ad..."
75,Tyler Leli,Research Assistant,To Whom It May Concern:I was thrilled to learn about the available Research Assistant positions with Rookieplay. I believe that I am a well-qualified candidate for this opportunity. Hopefully you ...,"Tyler Leli 227 Troy Avenue +1 919-995-4328 Brooklyn, New York ..."
76,Veronique Calmels,Research Assistant,I was thrilled when I saw a posting for a Research Assistant position. I have always been passionate about psychology and its biological influences. Throughout my college and professional career I...,"Veronique A. Calmels630.303.4976 | veronique834@gmail.comEducation BS Psychology | University of Denver - Denver, Colorado (8/2016-8/2019)· Major: Psychology- Cognitive Neuroscience concentration ..."


In [140]:
resumes_usernames = resumes['username']
resumes_raw_resume = resumes['raw_resume']

In [141]:
to_add_usernames = []
to_add_resumes = []

for username in resumes_usernames:
    if username not in list(compiled_df['username']):
        to_add_usernames.append(username)
for resume in resumes_raw_resume:
    if resume not in list(compiled_df['raw_resume']):
        to_add_resumes.append(resume)

In [142]:
len(to_add_usernames)

115

In [143]:
len(to_add_resumes)

115

In [144]:
to_add_df = pd.DataFrame(list(zip(to_add_usernames, to_add_resumes)), 
               columns =['username', 'raw_resume']) 

In [145]:
to_add_df['title']='Research Assistant'

In [146]:
to_add_df

,username,raw_resume,title
0,Abra Burkett,"Abra BurkettBrooklyn, NY 11206abraburkett3_pyd@indeedemail.com(310) 351-6405Authorized to work in the US for any employerWork ExperienceSeasonalSix Flags America - Bowie, MDMarch 2019 to PresentS...",Research Assistant
1,Abubakar Khan,A B U B A K A R M . K H A N CURRENT DESIGNATION Currently a Medical Graduate in the process of completing the USMLE. ECFMG Certification: -Step 1: PASSED (1st attempt) -Step 2: Pending EDUCA...,Research Assistant
2,Allora Atiyeh,"Allora AtiyehSchnecksville, PA 18078alloraatiyeh9_pn3@indeedemail.com(484) 951-6544Authorized to work in the US for any employerWork ExperienceLab TechnicianLaurie Drinkwater's Ecology Lab, Corne...",Research Assistant
3,Amena Rupani,"Amena RupaniAssistant Manager - Dunkin DonutsGlen Oaks, NY 11004amenarupani9_qbq@indeedemail.com646-242-1968Authorized to work in the US for any employerWork ExperienceAssistant ManagerDunkin Don...",Research Assistant
4,Amir Banai,"Amir BanaiNew York, NY 10001amirbanai6_7zi@indeedemail.com(703) 314-5775Work ExperienceTutorVarsity Tutors - New York, NYJune 2017 to Present• Taught college students Biochemistry, Biology, Engli...",Research Assistant
...,...,...,...
110,Yolanda Clark,"YOLANDA CLARKPROFESSIONAL SUMMARY· 5+ years high volume disaster relief call center, customer service, clerical/administrative support experience· Expert in customer care/communications, relations...",Research Assistant
111,YuanYuan Jiang,"YUANYUAN JIANG218-17 73 Ave., Unit 1 Bayside, NY, USA 11364 (347)213-5507 jyy2269@gmail.comPermanent Work Authorization • No Visa Sponsorship RequiredEDUCATION PhD in Marketing ...",Research Assistant
112,Yvonne Nyarko,"Yvonne Nyarko2160 E Tremont AveBronx, NY 10462646-633-9067Ynyarko2@gmail.comEDUCATIONNortheast Ohio Medical University (NEOMED)\t, Rootstown, OH\tGraduate Class of 2021· Pharmacy program Pharm.D.T...",Research Assistant
113,Zainab Mohammed,"Zainab MohammedINSTRUCTORSan Antonio, TX 78238zainabamohammed8_dtu@indeedemail.com210-842-6892Work ExperienceINSTRUCTORUNIVERSITY OF TEXAS SAN ANTONIOSeptember 2019 to Present6 credit hours • Pr...",Research Assistant


In [147]:
for row in to_add_df:
    compiled_df['username'].append(to_add_df.username)
    compiled_df['title'].append(to_add_df.title)
    compiled_df['raw_cover_letters'].append(None)
    compiled_df['raw_resume'].append(to_add_df.raw_resume)
    

In [148]:
compiled_df

,username,title,raw_cover_letters,raw_resume
0,Adaora Aroh,Research Assistant,"My name is Adaora Aroh and I am a recent master’s degree graduate in Biochemistry and Molecular Biology, I served as a student research assistant during my program and I have almost 2 years of res...","Adaora ArohOpen to the following job roles; Research assistant, Research Technician, ResearchAssociate, Clinical research coordinator, Project coordinatorAstoria, NY 11106adaoraaroh8_m5t@indeedem..."
1,Alec Getz,Research Assistant,"I’m writing to express my interest in applying for the position of research assistant (as posted on indeed.com). Although I don't have a sociology degree, I graduated cum laude in philosophy from ...","Alec GetzNew York, NY 10003alecgetz6_7o2@indeedemail.com917-657-1098Work ExperiencePractice Office AssociateFlexStaff - New York, NYSeptember 2019 to Present• Met with patients to acquire insuran..."
2,Ali Munk,Research Assistant,"Although I do not have exact experience for this position. In my previous role at Infinite CBD, I did lots of research on the human body, the endocannabinoid system, and how the cannabis plant in ...","Ali MunkContent Queen with a Business Savvy MindDenver, CO 80246allisonmunk4_hxx@indeedemail.com7193933732Motivated, young creative with proven experience in planning and executing events, digita..."
3,Amenaghawon Oyegun,Research Assistant,"Dear Rookieplay,The Research Assistant role with your organization aligns with my past career roles in research and public health. I am applying for this role because I value Rookieplay’s commitme...","Amenaghawon Oyegun Austin, Texas ▪ (682) 472-5689 ▪ sere1009@gmail.com\tProfessional ExperienceTexas Health and Human Services Civil Rights Office\t\t\t\t \t\tAustin, Texas 40 hou..."
4,Anna Ferguson,Research Assistant,"June 8, 2020RookiePlayNew York, New YorkTo whom it may concern,I am writing to you regarding the Research Assistant position currently available with Rookieplay. With a background in Anthropology,...","Anna FergusonLansdale, PA 19446annaferguson725_e4s@indeedemail.com2159332315Experienced Marketing Assistant with a demonstrated history of working in the individual and familyservices industry. S..."
...,...,...,...,...
73,Stephanie Simms,Research Assistant,"June 8, 2020To Whom It May Concern:I would like to express my interest at Rookieplay as a Research Assistant. As a graduate in Psychology and Spanish Language from James Madison University and a M...","Stephanie M. Simmssmsimms17@gmail.comEnglish and Spanish speakingEDUCATIONUniversidad de Málaga, October 2017 – October 2019· Master’s of Applied Sociology· Thesis: Analysis of Volunteers in the S..."
74,Tess Tucker,Research Assistant,"To Whom It May Concern,My name is Tess Tucker, and I have a strong background in executive office administration and management in a telecommuting environment. I love Rookieplay’s mission to help ...","Tess TuckerVirtual Executive Assistant & Marketing SpecialistPensacola, FL 32504tesstucker7_eqh@indeedemail.com(850) 898-4004I have 10+ years of experience in the customer service, managerial, ad..."
75,Tyler Leli,Research Assistant,To Whom It May Concern:I was thrilled to learn about the available Research Assistant positions with Rookieplay. I believe that I am a well-qualified candidate for this opportunity. Hopefully you ...,"Tyler Leli 227 Troy Avenue +1 919-995-4328 Brooklyn, New York ..."
76,Veronique Calmels,Research Assistant,I was thrilled when I saw a posting for a Research Assistant position. I have always been passionate about psychology and its biological influences. Throughout my college and professional career I...,"Veronique A. Calmels630.303.4976 | veronique834@gmail.comEducation BS Psychology | University of Denver - Denver, Colorado (8/2016-8/2019)· Major: Psychology- Cognitive Neuroscience concentration ..."


In [149]:
compiled_df = compiled_df.append(to_add_df)

In [150]:
compiled_df = compiled_df.reset_index(drop=True)

In [151]:
compiled_df

,username,title,raw_cover_letters,raw_resume
0,Adaora Aroh,Research Assistant,"My name is Adaora Aroh and I am a recent master’s degree graduate in Biochemistry and Molecular Biology, I served as a student research assistant during my program and I have almost 2 years of res...","Adaora ArohOpen to the following job roles; Research assistant, Research Technician, ResearchAssociate, Clinical research coordinator, Project coordinatorAstoria, NY 11106adaoraaroh8_m5t@indeedem..."
1,Alec Getz,Research Assistant,"I’m writing to express my interest in applying for the position of research assistant (as posted on indeed.com). Although I don't have a sociology degree, I graduated cum laude in philosophy from ...","Alec GetzNew York, NY 10003alecgetz6_7o2@indeedemail.com917-657-1098Work ExperiencePractice Office AssociateFlexStaff - New York, NYSeptember 2019 to Present• Met with patients to acquire insuran..."
2,Ali Munk,Research Assistant,"Although I do not have exact experience for this position. In my previous role at Infinite CBD, I did lots of research on the human body, the endocannabinoid system, and how the cannabis plant in ...","Ali MunkContent Queen with a Business Savvy MindDenver, CO 80246allisonmunk4_hxx@indeedemail.com7193933732Motivated, young creative with proven experience in planning and executing events, digita..."
3,Amenaghawon Oyegun,Research Assistant,"Dear Rookieplay,The Research Assistant role with your organization aligns with my past career roles in research and public health. I am applying for this role because I value Rookieplay’s commitme...","Amenaghawon Oyegun Austin, Texas ▪ (682) 472-5689 ▪ sere1009@gmail.com\tProfessional ExperienceTexas Health and Human Services Civil Rights Office\t\t\t\t \t\tAustin, Texas 40 hou..."
4,Anna Ferguson,Research Assistant,"June 8, 2020RookiePlayNew York, New YorkTo whom it may concern,I am writing to you regarding the Research Assistant position currently available with Rookieplay. With a background in Anthropology,...","Anna FergusonLansdale, PA 19446annaferguson725_e4s@indeedemail.com2159332315Experienced Marketing Assistant with a demonstrated history of working in the individual and familyservices industry. S..."
...,...,...,...,...
188,Yolanda Clark,Research Assistant,NaN,"YOLANDA CLARKPROFESSIONAL SUMMARY· 5+ years high volume disaster relief call center, customer service, clerical/administrative support experience· Expert in customer care/communications, relations..."
189,YuanYuan Jiang,Research Assistant,NaN,"YUANYUAN JIANG218-17 73 Ave., Unit 1 Bayside, NY, USA 11364 (347)213-5507 jyy2269@gmail.comPermanent Work Authorization • No Visa Sponsorship RequiredEDUCATION PhD in Marketing ..."
190,Yvonne Nyarko,Research Assistant,NaN,"Yvonne Nyarko2160 E Tremont AveBronx, NY 10462646-633-9067Ynyarko2@gmail.comEDUCATIONNortheast Ohio Medical University (NEOMED)\t, Rootstown, OH\tGraduate Class of 2021· Pharmacy program Pharm.D.T..."
191,Zainab Mohammed,Research Assistant,NaN,"Zainab MohammedINSTRUCTORSan Antonio, TX 78238zainabamohammed8_dtu@indeedemail.com210-842-6892Work ExperienceINSTRUCTORUNIVERSITY OF TEXAS SAN ANTONIOSeptember 2019 to Present6 credit hours • Pr..."


In [152]:
# Drops extra columns. We only want username and text columns
compiled_df = compiled_df.drop(compiled_df.columns[3], axis=1)

In [153]:
final_df = pd.read_csv('data/raw_data.csv', index_col=0)

In [154]:
final_df

,username,title,raw_cover_letters,raw_resume
0,Abdul Albab,Accounts Payable Specialist,NaN,"Abdul AlbabBronx, NY 10467abdulalbab2_n6u@indeedemail.com(347) 615-8170Work ExperienceAccountantCS Corporation - Bronx, NYJune 2017 to Present• Manage and oversee daily Accounts Payable and Accou..."
1,Abdul Shafaq,Accounts Payable Specialist,NaN,"Abdul Waheed Shafaq8581 N. Monroe Avenue, Lindenhurst, NY, 11757 | shafaqw2014@gmail.com | 347-335-8422Skills and Qualifications· Professional interpreter & translator; fluent in English, Dari, Pa..."
2,Abigail Riveron,Accounts Payable Specialist,"Dear Hiring Manager,I would like to express my deep interest in a position in your company.With my strong understanding of business and accounting principles gained throughout my education backgro...","CollegeGrad.com TemplateABIGAIL RIVERONMiami, FL 33177Mobile: 305.746.1662ariveron00@gmail.comProfessional SummaryEfficient, accurate and detail-oriented who takes initiative and has an innate dr..."
3,Abirami Suresh,Accounts Payable Specialist,NaN,"- Abirami Suresh.pagesAbirami Suresh 17 Bethel Lane, Schaumburg IL 60194 Mobile: 630-379-1405 Email: abisuresh62@gmail.com CAREER SUMMARY Highly self-motivated Senior Auditing Accountant with dem..."
4,Abria Bell,Training Manager,NaN,"Abria BellProject AdministratorChicago, IL 60617bellabria1213_2io@indeedemail.com7739700598#readytoworkWork ExperienceBilling / Administrative AssistantProject Org and Design Studio Inc - Chicago..."
...,...,...,...,...
329,Vincent Rullo,Media Planner,NaN,"Vincent Rullo 514 Bay Ridge Parkway Apt #2BBrooklyn, NY 11209Email: vinnierullo27@gmail.comPhone : 518-596-8359EMPLOYMENT HISTORYThe Topps Company ..."
330,Waid Harbison,Media Planner,NaN,"Waid HarbisonCullman, AL 35055waid85_atc@indeedemail.com256-708-1650Work ExperiencePresident & Chief Marketing OfficerA-1 Marketing Services LLCApril 2019 to PresentStarted own business working w..."
331,Yasmina Ouedraogo,Media Planner,NaN,"Yasmina OuedraogoBrooklyn, NY 11207yasminaouedraogo9_k5n@indeedemail.com3478630020Work ExperienceInternNew York Stage Supreme Court - Brooklyn, NYJuly 2019 to August 2019Observed court proceeding..."
332,Yayonah Bangura,Media Planner,NaN,YAYONAH BANGURAPage Two of TwoYAYONAH BANGURA (732) 421-4186 Yayonahb@gmail.com www.linkedin.com/in/yayonahbWebsite: theaknown.com/features-1PROFILEAmbitious and motivated communications special...


In [155]:
final_df = final_df.append(compiled_df)

In [156]:
final_df = final_df.reset_index(drop=True)

In [157]:
final_df.to_csv('data/raw_data.csv')

## Check for duplicates

In [158]:
final_df = pd.read_csv(r'C:\Users\sambe\Projects\Cover_Letter_Analysis\data\raw_data.csv', index_col=0)

In [159]:
final_df = final_df.sort_values('username').reset_index(drop=True)

In [160]:
final_df.to_csv('data/raw_data.csv')

In [161]:
final_df

,username,title,raw_cover_letters,raw_resume
0,Aakash Patel,Media Planner,"I was excited to discover the Media Planner position with Rookieplay, which I saw posted on Indeed. Rookieplay is a company on the rise, and as a recent college graduate with 2 years of diverse ma...","Microsoft Word - Aakash Patel 1\tAakash Patel aakashp047@gmail.com v (516) 404-6119 v Syosset, NY EDUCATION Stony Brook University ..."
1,Aaron Neal,Media Planner,"Dear Hiring Manager,I am writing to you today to apply for the open position of Media Planner. As a marketing professional with 2+ years of experience I believe that I will be an asset to the Rook...","_Aaron Neal_052020_v2A A R O N M . N E A LSTRATEGISTPERSONALTRAITS & SKILLSEDUCATIONCONTACT770.871.5294aaronm.neal3@gmail.comProficient in Simmons OneView, Global Web Index MS Office Perspective..."
2,Aaron Penzel,Media Planner,"Dear Hiring Manager,I am writing to express interest in Rookieplay’s Media Planner position. I am a creative and out-of-the-box marketer in the entertainment industry, with experience in both non-...","AARON PENZELAARON PENZELENTERTAINMENT ADVERTISERCONTACTPHONE: (908) 217-7289 EMAIL: A.PENZEL724@GMAIL.COM 23-43 28TH STREET LONG ISLAND CITY, NY 11105 WORK EXPERIENCEManaged all traditional media..."
3,Abby Cleary,Media Planner,NaN,"Abby ClearyBuffalo, NYabbycleary7_pb3@indeedemail.com(716) 238-0440Authorized to work in the US for any employerWork ExperienceInternSwanson Communications - New York, NYJune 2019 to Present• Ass..."
4,Abdul Albab,Accounts Payable Specialist,NaN,"Abdul AlbabBronx, NY 10467abdulalbab2_n6u@indeedemail.com(347) 615-8170Work ExperienceAccountantCS Corporation - Bronx, NYJune 2017 to Present• Manage and oversee daily Accounts Payable and Accou..."
...,...,...,...,...
2647,Zihao Zhou,Accounts Payable Specialist,NaN,"Zihao ZhouManager / Business Development DirectorBoston, MALike challenging new tasks. Positive personality. Good at communication, experienced marketing andproducts specialist.Willing to relocat..."
2648,Zira Cortes,Training Manager,NaN,"Zira CortesSubstitute Teacher - SANDRIDGE Elementary SchoolChicago, IL 60633ziracortes9_rew@indeedemail.com(773)-704-4172I am seeking a full-time position where I can use my experience and educat..."
2649,Zoya Buyankina,Senior Account Manager,NaN,"ZOYA BUYANKINA SUMMARY: A young, self-motivated and achievement-oriented Business Development Analyst with more than 3 years of experience in the business development process and different marketi..."
2650,Zoya Siddiqui,Research Assistant,NaN,NaN


In [162]:
test_df = final_df

In [163]:
test_df

,username,title,raw_cover_letters,raw_resume
0,Aakash Patel,Media Planner,"I was excited to discover the Media Planner position with Rookieplay, which I saw posted on Indeed. Rookieplay is a company on the rise, and as a recent college graduate with 2 years of diverse ma...","Microsoft Word - Aakash Patel 1\tAakash Patel aakashp047@gmail.com v (516) 404-6119 v Syosset, NY EDUCATION Stony Brook University ..."
1,Aaron Neal,Media Planner,"Dear Hiring Manager,I am writing to you today to apply for the open position of Media Planner. As a marketing professional with 2+ years of experience I believe that I will be an asset to the Rook...","_Aaron Neal_052020_v2A A R O N M . N E A LSTRATEGISTPERSONALTRAITS & SKILLSEDUCATIONCONTACT770.871.5294aaronm.neal3@gmail.comProficient in Simmons OneView, Global Web Index MS Office Perspective..."
2,Aaron Penzel,Media Planner,"Dear Hiring Manager,I am writing to express interest in Rookieplay’s Media Planner position. I am a creative and out-of-the-box marketer in the entertainment industry, with experience in both non-...","AARON PENZELAARON PENZELENTERTAINMENT ADVERTISERCONTACTPHONE: (908) 217-7289 EMAIL: A.PENZEL724@GMAIL.COM 23-43 28TH STREET LONG ISLAND CITY, NY 11105 WORK EXPERIENCEManaged all traditional media..."
3,Abby Cleary,Media Planner,NaN,"Abby ClearyBuffalo, NYabbycleary7_pb3@indeedemail.com(716) 238-0440Authorized to work in the US for any employerWork ExperienceInternSwanson Communications - New York, NYJune 2019 to Present• Ass..."
4,Abdul Albab,Accounts Payable Specialist,NaN,"Abdul AlbabBronx, NY 10467abdulalbab2_n6u@indeedemail.com(347) 615-8170Work ExperienceAccountantCS Corporation - Bronx, NYJune 2017 to Present• Manage and oversee daily Accounts Payable and Accou..."
...,...,...,...,...
2647,Zihao Zhou,Accounts Payable Specialist,NaN,"Zihao ZhouManager / Business Development DirectorBoston, MALike challenging new tasks. Positive personality. Good at communication, experienced marketing andproducts specialist.Willing to relocat..."
2648,Zira Cortes,Training Manager,NaN,"Zira CortesSubstitute Teacher - SANDRIDGE Elementary SchoolChicago, IL 60633ziracortes9_rew@indeedemail.com(773)-704-4172I am seeking a full-time position where I can use my experience and educat..."
2649,Zoya Buyankina,Senior Account Manager,NaN,"ZOYA BUYANKINA SUMMARY: A young, self-motivated and achievement-oriented Business Development Analyst with more than 3 years of experience in the business development process and different marketi..."
2650,Zoya Siddiqui,Research Assistant,NaN,NaN


In [22]:
test_df['username'][27]

'Alahree Martin'

In [164]:
test_df.drop_duplicates(subset ="username", 
                     keep = 'first', inplace = True) 
      

In [165]:
test_df

,username,title,raw_cover_letters,raw_resume
0,Aakash Patel,Media Planner,"I was excited to discover the Media Planner position with Rookieplay, which I saw posted on Indeed. Rookieplay is a company on the rise, and as a recent college graduate with 2 years of diverse ma...","Microsoft Word - Aakash Patel 1\tAakash Patel aakashp047@gmail.com v (516) 404-6119 v Syosset, NY EDUCATION Stony Brook University ..."
1,Aaron Neal,Media Planner,"Dear Hiring Manager,I am writing to you today to apply for the open position of Media Planner. As a marketing professional with 2+ years of experience I believe that I will be an asset to the Rook...","_Aaron Neal_052020_v2A A R O N M . N E A LSTRATEGISTPERSONALTRAITS & SKILLSEDUCATIONCONTACT770.871.5294aaronm.neal3@gmail.comProficient in Simmons OneView, Global Web Index MS Office Perspective..."
2,Aaron Penzel,Media Planner,"Dear Hiring Manager,I am writing to express interest in Rookieplay’s Media Planner position. I am a creative and out-of-the-box marketer in the entertainment industry, with experience in both non-...","AARON PENZELAARON PENZELENTERTAINMENT ADVERTISERCONTACTPHONE: (908) 217-7289 EMAIL: A.PENZEL724@GMAIL.COM 23-43 28TH STREET LONG ISLAND CITY, NY 11105 WORK EXPERIENCEManaged all traditional media..."
3,Abby Cleary,Media Planner,NaN,"Abby ClearyBuffalo, NYabbycleary7_pb3@indeedemail.com(716) 238-0440Authorized to work in the US for any employerWork ExperienceInternSwanson Communications - New York, NYJune 2019 to Present• Ass..."
4,Abdul Albab,Accounts Payable Specialist,NaN,"Abdul AlbabBronx, NY 10467abdulalbab2_n6u@indeedemail.com(347) 615-8170Work ExperienceAccountantCS Corporation - Bronx, NYJune 2017 to Present• Manage and oversee daily Accounts Payable and Accou..."
...,...,...,...,...
2647,Zihao Zhou,Accounts Payable Specialist,NaN,"Zihao ZhouManager / Business Development DirectorBoston, MALike challenging new tasks. Positive personality. Good at communication, experienced marketing andproducts specialist.Willing to relocat..."
2648,Zira Cortes,Training Manager,NaN,"Zira CortesSubstitute Teacher - SANDRIDGE Elementary SchoolChicago, IL 60633ziracortes9_rew@indeedemail.com(773)-704-4172I am seeking a full-time position where I can use my experience and educat..."
2649,Zoya Buyankina,Senior Account Manager,NaN,"ZOYA BUYANKINA SUMMARY: A young, self-motivated and achievement-oriented Business Development Analyst with more than 3 years of experience in the business development process and different marketi..."
2650,Zoya Siddiqui,Research Assistant,NaN,NaN


In [ ]:
baby_test_df = 